In [1]:
from google.colab import drive 
drive.mount('/content/drive/')


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
!ls /content/drive/MyDrive/Colab/

 639.ipynb	 graph	     list_patition_label.txt		 Untitled
 final_project	 helper.py  'processed_data_(100, 100).pickle'	 vgg.py


In [3]:
root = '/content/drive/MyDrive/Colab/'

In [4]:
def label_loader(label_file):
    train_img_label_pair = {}
    test_img_label_pair ={}
    with open(label_file) as all_label:
        for label in all_label:
            label = label.rstrip()
            if label.startswith("train"):
                train_img_label_pair[label.split(" ")[0][:-4]+"_aligned.jpg"] = int(label.split(" ")[1])
            if label.startswith("test"):
                test_img_label_pair[label.split(" ")[0][:-4]+"_aligned.jpg"] = int(label.split(" ")[1])
    return train_img_label_pair, test_img_label_pair

In [5]:
def plot(history, log_name, num_epoch):
    """
    plot accuracy and loss save to "./imgs"
    :param history: tensorflow History Object
    :param log_name: String:name of the log
    :param num_epoch: int: number of epoches
    """
    import matplotlib.pyplot as plt
    import os
    if not os.path.exists(root + 'graph'):
        print("First run, make dir")
        os.makedirs(root + 'graph')
    plt.plot(np.linspace(1, num_epoch, num_epoch),
             np.array(history.history["categorical_accuracy"]), label='Training Accuracy', color='b')
    plt.plot(np.linspace(1, num_epoch, num_epoch),
             np.array(history.history["val_categorical_accuracy"]), label='Test Accuracy', color='r')
    plt.legend()
    plt.title("Accuracy" + log_name + time.ctime())
    plt.savefig(root +"graph/Accuracy " + log_name + " " + time.ctime())
    # plt.show()
    plt.close()
    plt.plot(np.linspace(1, num_epoch, num_epoch), np.array(history.history["loss"]), label='Training Loss', color='b')
    plt.plot(np.linspace(1, num_epoch, num_epoch),
             np.array(history.history["val_loss"]), label='Test Loss', color='r')
    plt.legend()
    plt.title("Loss" + log_name + time.ctime())
    plt.savefig(root +"graph/Loss " + log_name + " " + time.ctime())
    # plt.show()
    plt.close()

In [6]:



num_epoch = 10 #20
batch_size = 64
input_shape = (100, 100)


# log_name = input("What's the name of this run?:")

# create image label pair as a dictionary
root = '/content/drive/MyDrive/Colab/'
label_file = root + 'list_patition_label.txt'
# folder = root +'/RAFDB/aligned/'
train_img_label_pair, test_img_label_pair = label_loader(label_file)

# Use pickle to speed up reading data
import pickle
print("Loading Data")
try:
    train_img, train_label, test_img, test_label = pickle.load(open(root + 'processed_data_(100, 100).pickle', 'rb'))
    print("Shape", train_img.shape)
except:
    print("hi")
    # train_img, train_label = helper.load_to_numpy(train_img_label_pair, folder, input_shape)
    # test_img, test_label = helper.load_to_numpy(test_img_label_pair, folder, input_shape)
    # print("Shape", train_img.shape)
    # print("First time reading data, saving to pickle to speed up next time")
    # try:
    #     pickle.dump((train_img, train_label, test_img, test_label), open("processed_data_"+str(input_shape)+".pickle", 'wb'))
    # except:
    #     print("can't save, file too big")


import os
if not os.path.exists("./trained_model"):
    print("First run, make model dir")
    os.makedirs("./trained_model")




Loading Data
Shape (12271, 100, 100, 3)


In [9]:
# Keras model and log
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
import time
import numpy as np
from tensorflow.keras import layers, Sequential
import tensorflow as tf
from tensorflow.keras import regularizers
num_epoch = 60 #20
batch_size = 64

log_name = input("What's the name of this run?:")

model = Sequential([
        # Block 1: in:100x100x3, out:50x50x8
        layers.Conv2D(8, (3, 3), activation='relu', padding='same', name='b1_conv1'),
        layers.Conv2D(8, (3, 3), activation='relu', padding='same', name='b1_conv2',kernel_regularizer =tf.keras.regularizers.l1( l=0.01) ),
        layers.MaxPooling2D((2, 2), strides=(2, 2), name='b1_maxpool'),

        # Block 2 in:50x50x8, out:25x25x16
        layers.Conv2D(16, (3, 3), activation='relu', padding='same', name='b2_conv1'),
        layers.Conv2D(16, (3, 3), activation='relu', padding='same', name='b2_conv2',kernel_regularizer =tf.keras.regularizers.l1( l=0.01) ),
        layers.MaxPooling2D((2, 2), strides=(2, 2), name='b2_maxpool'),

        # Blcok 3 in:25x25x16, out:12x12x32
        # layers.Conv2D(32, (3, 3), activation='relu', padding='same', name='b3_conv1'),
        # layers.Conv2D(32, (3, 3), activation='relu', padding='same', name='b3_conv2'),
        # layers.MaxPooling2D((2, 2), strides=(2, 2), name='b3_maxpool'),

        # Full connect layer

        layers.Flatten(name='flatten'),
        layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.001),name='fc1'),
        layers.Dropout(0.7, seed=42),  # Answer to the Ultimate Question of Life, the Universe, and Everything (42)
        layers.Dense(512, activation='relu', name='fc2',kernel_regularizer=regularizers.l2(0.001)),
        layers.Dropout(0.7, seed=42),
        layers.Dense(64, activation='relu', name='fc4'),

        layers.Dense(7, activation='softmax')
    ])
opt = tf.keras.optimizers.RMSprop(learning_rate=0.0001)
model.compile(optimizer=opt,
                  loss='categorical_crossentropy',
                  metrics=[tf.keras.metrics.categorical_accuracy])


tb = TensorBoard(log_dir="logs/" + log_name +" "+ time.ctime())
chk = ModelCheckpoint("./trained_model/"+log_name+"_chk.h5", monitor='val_categorical_accuracy', verbose=0,
                      save_best_only=True, save_weights_only=True, mode='max', period=1)
history = model.fit(train_img, train_label, epochs=num_epoch, batch_size=batch_size,
                    validation_data=(test_img, test_label), callbacks=[tb, chk],)
model.summary()
plot(history, log_name, num_epoch)

model.save("./trained_model/"+log_name+".h5")
loss, accuracy = model.evaluate(test_img, test_label,batch_size=32)
print("test loss:", loss)
print("test accuracy:", accuracy)



What's the name of this run?:128
Epoch 1/60
96/96 [==============================] - 5s 56ms/step - loss: 4.9383 - categorical_accuracy: 0.3739 - val_loss: 4.4604 - val_categorical_accuracy: 0.3862
Epoch 2/60
96/96 [==============================] - 4s 37ms/step - loss: 4.2058 - categorical_accuracy: 0.3856 - val_loss: 3.9324 - val_categorical_accuracy: 0.3862
Epoch 3/60
96/96 [==============================] - 4s 37ms/step - loss: 3.7567 - categorical_accuracy: 0.3886 - val_loss: 3.5651 - val_categorical_accuracy: 0.3862
Epoch 4/60
96/96 [==============================] - 4s 38ms/step - loss: 3.4255 - categorical_accuracy: 0.3897 - val_loss: 3.2602 - val_categorical_accuracy: 0.3869
Epoch 5/60
96/96 [==============================] - 4s 38ms/step - loss: 3.1475 - categorical_accuracy: 0.3930 - val_loss: 3.0009 - val_categorical_accuracy: 0.3921
Epoch 6/60
96/96 [==============================] - 4s 39ms/step - loss: 2.9121 - categorical_accuracy: 0.3992 - val_loss: 2.7797 - val_catego

In [8]:
!kill 378

/bin/bash: line 0: kill: (378) - No such process
